In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DS-CodingDojoColab/Project2/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv')
df.head()

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
0,61639,10025,1,"New York, NY",3025.0,3039.0,3054.0,3068.0,3083.0,3097.0,3112.0,3126.0,3140.0,3155.0,3169.0,3182.0,3196.0,3207.0,3217.0,3227.0,3233.0,3239.0,3244.0,3248.0,3252.0,3257.0,3260.0,3264.0,3268.0,3270.0,3272.0,3274.0,3276.0,3277.0,3279.0,3277.0,3276.0,3275.0,3275.0,3274.0,...,3267.0,3264.0,3263.0,3261.0,3260.0,3260.0,3260.0,3260.0,3262.0,3264.0,3267.0,3270.0,3274.0,3277.0,3281.0,3286.0,3290.0,3298.0,3305.0,3313.0,3324.0,3334.0,3345.0,3353.0,3361.0,3369.0,3361.0,3353.0,3346.0,3311.0,3277.0,3242.0,3201.0,3159.0,3117.0,3073.0,3029.0,2985.0,2938.0,2891.0
1,84654,60657,2,"Chicago, IL",1580.0,1585.0,1591.0,1596.0,1602.0,1607.0,1612.0,1618.0,1623.0,1628.0,1633.0,1638.0,1643.0,1648.0,1653.0,1657.0,1662.0,1666.0,1670.0,1675.0,1679.0,1684.0,1689.0,1694.0,1699.0,1703.0,1708.0,1713.0,1717.0,1722.0,1726.0,1730.0,1735.0,1739.0,1742.0,1745.0,...,1761.0,1762.0,1763.0,1765.0,1766.0,1766.0,1766.0,1767.0,1767.0,1767.0,1767.0,1768.0,1769.0,1770.0,1773.0,1776.0,1778.0,1783.0,1788.0,1793.0,1799.0,1806.0,1812.0,1817.0,1822.0,1827.0,1828.0,1829.0,1830.0,1825.0,1820.0,1816.0,1810.0,1803.0,1797.0,1791.0,1784.0,1777.0,1770.0,1762.0
2,61637,10023,3,"New York, NY",3156.0,3164.0,3172.0,3180.0,3188.0,3195.0,3203.0,3210.0,3218.0,3225.0,3231.0,3237.0,3242.0,3248.0,3254.0,3260.0,3267.0,3274.0,3281.0,3287.0,3294.0,3300.0,3305.0,3309.0,3314.0,3314.0,3313.0,3313.0,3309.0,3305.0,3301.0,3298.0,3295.0,3292.0,3293.0,3295.0,...,3304.0,3303.0,3300.0,3298.0,3296.0,3295.0,3295.0,3295.0,3299.0,3303.0,3306.0,3310.0,3314.0,3317.0,3323.0,3328.0,3333.0,3341.0,3350.0,3358.0,3366.0,3375.0,3384.0,3391.0,3397.0,3404.0,3397.0,3390.0,3383.0,3349.0,3316.0,3282.0,3241.0,3201.0,3160.0,3116.0,3073.0,3029.0,2983.0,2937.0
3,91982,77494,4,"Houston, TX",1778.0,1781.0,1784.0,1787.0,1790.0,1793.0,1795.0,1798.0,1801.0,1803.0,1805.0,1806.0,1808.0,1807.0,1807.0,1807.0,1805.0,1804.0,1802.0,1799.0,1796.0,1792.0,1787.0,1782.0,1777.0,1770.0,1764.0,1758.0,1751.0,1744.0,1738.0,1733.0,1729.0,1724.0,1722.0,1720.0,...,1735.0,1739.0,1742.0,1745.0,1747.0,1749.0,1750.0,1751.0,1751.0,1751.0,1751.0,1752.0,1753.0,1753.0,1755.0,1757.0,1759.0,1762.0,1764.0,1767.0,1769.0,1771.0,1773.0,1775.0,1776.0,1778.0,1779.0,1780.0,1781.0,1783.0,1785.0,1787.0,1791.0,1794.0,1797.0,1800.0,1804.0,1807.0,1810.0,1814.0
4,84616,60614,5,"Chicago, IL",1704.0,1709.0,1714.0,1720.0,1724.0,1729.0,1734.0,1738.0,1743.0,1747.0,1752.0,1756.0,1761.0,1766.0,1771.0,1776.0,1782.0,1787.0,1793.0,1801.0,1809.0,1818.0,1825.0,1833.0,1840.0,1846.0,1851.0,1857.0,1861.0,1864.0,1868.0,1871.0,1874.0,1876.0,1878.0,1880.0,...,1902.0,1905.0,1908.0,1910.0,1912.0,1914.0,1915.0,1917.0,1918.0,1919.0,1921.0,1923.0,1926.0,1928.0,1932.0,1935.0,1939.0,1945.0,1950.0,1956.0,1962.0,1969.0,1975.0,1979.0,1984.0,1988.0,1990.0,1991.0,1993.0,1986.0,1980.0,1973.0,1963.0,1953.0,1943.0,1931.0,1920.0,1909.0,1897.0,1885.0


In [5]:
df.shape

(2263, 88)